# Batch Inference

In [20]:
import boto3
import os
import pandas as pd
import numpy as np
import time
import sagemaker
import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

In [21]:
%env AWS_PROFILE=aeroxye-sagemaker

env: AWS_PROFILE=aeroxye-sagemaker


In [22]:
!aws sts get-caller-identity

{
    "UserId": "AROAWC4YSIQL5OBFCNGEX:botocore-session-1687157949",
    "Account": "418542404631",
    "Arn": "arn:aws:sts::418542404631:assumed-role/SageMaker-UserRole/botocore-session-1687157949"
}


In [23]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SageMaker-UserRole')['Role']['Arn']

region = boto3.Session().region_name
print(f'Current region: {region}')

boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)
default_bucket = sagemaker_session.default_bucket()
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
sagemaker_client.list_feature_groups()

featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Current region: ap-southeast-1


## Download Model from Model Registry

In [24]:
model_package_group_name = 'TestModelPackageGroup'

In [25]:
from inference.utils import get_approved_package

pck = get_approved_package(model_package_group_name)
model_description = sagemaker_client.describe_model_package(ModelPackageName=pck["ModelPackageArn"])

INFO:utils:Identified the latest approved model package: arn:aws:sagemaker:ap-southeast-1:418542404631:model-package/testmodelpackagegroup/19


In [26]:
from sagemaker import ModelPackage

model_package_arn = model_description["ModelPackageArn"]
model = ModelPackage(role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session)

In [ ]:
# endpoint_name = "test-endpoint-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
# print("EndpointName= {}".format(endpoint_name))
# model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name)

## Run Batch Inference

### Pull user and interactions data from feature store

In [8]:
users_feature_group_name = 'users-feature-group'
users_feature_group = FeatureGroup(name=users_feature_group_name, sagemaker_session=sagemaker_session)

interactions_feature_group_name = 'interactions-feature-group'
interactions_feature_group = FeatureGroup(name=interactions_feature_group_name, sagemaker_session=sagemaker_session)

In [9]:
from sagemaker.feature_store.feature_store import FeatureStore

feature_store = FeatureStore(feature_store_session)
builder = feature_store.create_dataset(
    base=users_feature_group,
    output_path=f's3://{default_bucket}/batch-inference/store'
).with_feature_group(
    interactions_feature_group,
    target_feature_name_in_base="id",
    included_feature_names=["userID", "catID"],
    feature_name_in_target="userID"
)

In [10]:
df, query = builder.to_dataframe()

In [11]:
df

,id,has_other_cats,personality,gender,good_with_other_dogs,employment,created_at,agree_to_fee,is_first_cat,good_with_kids,...,age_senior,primary_color_no_preference,primary_color_black,primary_color_calico_tortie,primary_color_tabby,primary_color_others,primary_color_ginger,primary_color_white,userID.1,catID.1
0,be0db670-e5a7-4fb2-a480-0a8353562b63,0,anything is nice,no preference,0,working full time,2023-05-19T09:59:14Z,1,1,0,...,0,0,0,0,0,0,1,0,be0db670-e5a7-4fb2-a480-0a8353562b63,2c5fcdc6-9d32-4635-81da-8ac6afb760ee
1,be0db670-e5a7-4fb2-a480-0a8353562b63,0,anything is nice,no preference,0,working full time,2023-05-19T09:59:14Z,1,1,0,...,0,0,0,0,0,0,1,0,be0db670-e5a7-4fb2-a480-0a8353562b63,ce290f5f-6b15-4831-98d0-6a003a2a0b04
2,be0db670-e5a7-4fb2-a480-0a8353562b63,0,anything is nice,no preference,0,working full time,2023-05-19T09:59:14Z,1,1,0,...,0,0,0,0,0,0,1,0,be0db670-e5a7-4fb2-a480-0a8353562b63,9d28cf64-20f1-4628-9a96-9df502af03ef
3,be0db670-e5a7-4fb2-a480-0a8353562b63,0,anything is nice,no preference,0,working full time,2023-05-19T09:59:14Z,1,1,0,...,0,0,0,0,0,0,1,0,be0db670-e5a7-4fb2-a480-0a8353562b63,16ebe31c-cc7a-4b06-a3c5-3a8beba9946c
4,be0db670-e5a7-4fb2-a480-0a8353562b63,0,anything is nice,no preference,0,working full time,2023-05-19T09:59:14Z,1,1,0,...,0,0,0,0,0,0,1,0,be0db670-e5a7-4fb2-a480-0a8353562b63,7b1632fa-6152-4862-b9d9-3bd9207cf25c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4836,670c923b-34c4-4f0b-a147-600eaac8b793,0,all sweet,male,1,working full time,2023-06-06T02:23:53Z,1,1,1,...,0,0,0,0,1,0,0,0,670c923b-34c4-4f0b-a147-600eaac8b793,5b44494c-c2b6-47cb-b823-879b3b3d150d
4837,670c923b-34c4-4f0b-a147-600eaac8b793,0,all sweet,male,1,working full time,2023-06-06T02:23:53Z,1,1,1,...,0,0,0,0,1,0,0,0,670c923b-34c4-4f0b-a147-600eaac8b793,4bc6b512-595c-4429-8cf8-f945107fc9a9
4838,670c923b-34c4-4f0b-a147-600eaac8b793,0,all sweet,male,1,working full time,2023-06-06T02:23:53Z,1,1,1,...,0,0,0,0,1,0,0,0,670c923b-34c4-4f0b-a147-600eaac8b793,9b85f28e-98cc-48d9-aecf-225c40db506d
4839,670c923b-34c4-4f0b-a147-600eaac8b793,0,all sweet,male,1,working full time,2023-06-06T02:23:53Z,1,1,1,...,0,0,0,0,1,0,0,0,670c923b-34c4-4f0b-a147-600eaac8b793,0b34f316-6234-4eca-82a5-eca4ce167fbf


In [12]:
input_data = df.groupby('id')['catID.1'].apply(list).reset_index(name='seen')
input_data = input_data.rename(columns={'id': 'userID'})
input_data

,userID,seen
0,0153cd06-7ebb-41cc-b304-7b4e42c6b965,"[993c537d-c4e3-4343-854a-b36068506d8e, fd8e517..."
1,01abb157-9112-47c6-9ff6-fb603505e341,"[fd8e5177-f8cc-436a-ae82-a473ca5b4cc2, 7ef6aee..."
2,029f1e2a-d68b-4801-869f-f422e925ae3d,"[ff92ed1e-3259-45d6-9d95-fa10da0de0b7, 5bc42ab..."
3,079b0ec9-cec6-42fb-9f00-7891c52a10fb,"[2b87337b-1179-42c8-b423-c0ec34bb6833, c5c8e96..."
4,081f358f-9624-468a-b6ef-f9b9fad2b3b3,"[3fd9c274-509b-4283-a9c8-832ae290c247, 2b87337..."
...,...,...
99,eaf7de2b-dde2-4eca-9172-a720201a3c90,"[6e104908-8c00-45cd-ae3d-279dda0c3d9c, fd8e517..."
100,f3ff9f65-68ff-4333-a442-27d58f75d5ee,[9d075140-be62-4d86-bc13-19fc8ffd07e8]
101,fab5c53d-493a-4baf-b8e5-3cd1e505d75e,"[979dd476-808d-409d-aa6b-3bac02039b8d, 7c6f9a2..."
102,fafff5de-efc6-42f2-85b9-0b9c79a5abaa,"[f7521ce1-248a-4f2b-9800-4306b90966e4, f275533..."


In [13]:
data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok=True)

input_data.to_csv("./data/input_data.csv", header=False, index=False)

In [14]:
# output to s3
s3_prefix = "batch-inference/input-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
input_s3 = sagemaker_session.upload_data(path="./data", key_prefix=s3_prefix)

### Instantiate transformer

In [27]:
# create transformer
rank_file_name = f'ranking-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
output_path = f's3://petfinder6000-ranking/{rank_file_name}'
transformer = model.transformer(instance_count=1,
    instance_type='ml.m5.xlarge',
    assemble_with='Line',
    output_path=output_path)

INFO:sagemaker:Creating model with name: testmodelpackagegroup-2023-06-19-07-22-31-735


### Run batch inference job

In [28]:
# do batch transform
transformer.transform(data=input_s3, split_type='Line', content_type='text/csv')

INFO:sagemaker:Creating transform job with name: testmodelpackagegroup-2023-06-19-07-22-36-100


.....................
Starting the inference server with 4 workers.
[2023-06-19 07:26:09 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2023-06-19 07:26:09 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2023-06-19 07:26:09 +0000] [10] [INFO] Using worker: sync
[2023-06-19 07:26:09 +0000] [12] [INFO] Booting worker with pid: 12
[2023-06-19 07:26:09 +0000] [13] [INFO] Booting worker with pid: 13
[2023-06-19 07:26:09 +0000] [14] [INFO] Booting worker with pid: 14
[2023-06-19 07:26:09 +0000] [15] [INFO] Booting worker with pid: 15
b'ldd (Debian GLIBC 2.36-9) 2.36\nCopyright (C) 2022 Free Software Foundation, Inc.\nThis is free software; see the source for copying conditions.  There is NO\nwarranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.\nWritten by Roland McGrath and Ulrich Drepper.\n'
b'ldd (Debian GLIBC 2.36-9) 2.36\nCopyright (C) 2022 Free Software Foundation, Inc.\nThis is free software; see the source for copying conditions.  There is NO\nwarra

In [29]:
import re
s3client = boto3.client('s3')
def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    print(output_bucket, output_prefix)
    s3client.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=0)

In [30]:
transformer.output_path

's3://petfinder6000-ranking/ranking-2023-06-19-07-22-31'

In [31]:
batch_file_name = "input_data.csv"
output_df = get_csv_output_from_s3(transformer.output_path, batch_file_name)

petfinder6000-ranking ranking-2023-06-19-07-22-31/input_data.csv.out


In [32]:
output_df.loc[0, 'reco']

'2b87337b-1179-42c8-b423-c0ec34bb6833,30c033c1-9ebf-47de-ba85-bb2ca1029ed9,66fcc141-e657-4732-99e2-003e577ca7d1,f275533f-0bc5-495a-abfa-e193ab062849,72907ede-9f87-4008-be26-5a06bf50d87b,20742dae-b5f4-4c5e-a401-b71e60539f10,b3f07f4b-ef30-4433-919b-480f9d957a31,9c509932-95bd-4b2a-b10b-072c0b084fcd,3251d872-21b1-4934-ba86-5439045ab89b,41ede236-7ada-4f4f-845c-35a58d9c08d3,66c72d11-c009-4ff1-b976-353aa1072cac,a39a1f1f-2c4d-4ce4-801a-f63d900895dd,d73a6786-fac6-42a1-aa81-50c6a8618592,3fd9c274-509b-4283-a9c8-832ae290c247,6b33d102-9497-43c7-a7a3-a28d6a9d1880,554577ae-c5a2-414e-9834-e4e0f27efcb9,2c7e97d0-82df-4b28-a787-8727de88b163,80355278-da9b-4f01-9377-8fa82eefcbb5,20106676-a088-44e3-b23e-286b6e37f1d5,93c90dec-7f79-435d-bf3c-d9516cea96e7,adccdee5-8413-4344-8cca-13a99837a6a4,0b34f316-6234-4eca-82a5-eca4ce167fbf,ead698c7-eb09-4c7c-a322-b83bf3ca2126,50621fa7-de81-4141-9c19-e21844d8268f,3394e37a-7a56-4229-892e-d915a8b8921e,2a7b002c-7c5d-4b0e-8155-e75f42b0d449,bd8c1b25-bb4b-4d0d-ac37-3d6799d885f9,

## Cleanup

In [ ]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name=endpoint_name)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()